baseline.ipynb<br>
.. └ data<br>
.... ├ train.json<br>
.... ├ test.json<br>
.... └ sample_submission.csv<br>

# 사용 패키지

In [2]:
# ### 서버(우분투)에 KoNLPy 설치하기: https://konlpy.org/ko/v0.5.2/install/

# ### 1. jdk 설치
# sudo apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl

# ### 2. konlpy 설치
# python3 -m pip install --upgrade pip
# python3 -m pip install konlpy

# ### 3. Mecab 설치
# sudo apt-get install curl git
# bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

### module 'tweepy' has no attribute 'StreamListener' 에러 발생 시
- 아래 출력문으로 우선 tweepy 패키지가 설치되었는지 확인
- print(tweepy.__version__)  # '4.10.0'

설치되어 있다면, StreamListener 클래스가 Tweepy 버전4부터 Stream이란 클래스로 통합되어서 발생한 에러 [(참고)](https://docs.tweepy.org/en/stable/faq.html?highlight=streamlistener#where-did-streamlistener-go)
- 버전 3 중 가장 최신 버전인 3.10.0으로 Tweepy의 버전을 낮춘다.
- pip install tweepy==3.10.0

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import random
import math
import copy
import gc
from tqdm import tqdm
from glob import glob
from konlpy.tag import Mecab

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings(action='ignore')

## 랜덤 시드 고정

In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

In [3]:
seed_everything(42)

# 데이터 로드

In [4]:
DIR = "../data/raw_data"
TRAIN_SOURCE = os.path.join(DIR, "train.json")

In [5]:
with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())

In [6]:
train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            # context += ' '
            context += '\n\n'  # 태깅할 때 보기 편하게 개행 2번
            
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

In [8]:
def get_total_id(full_df):
    total_id = []
    meeting_id = 1000
    agenda_id = 1

    for i in range(len(full_df)-1):     
        if len(str(agenda_id)) == 1:
            id = str(meeting_id) + '0' + str(agenda_id)
        else:
            id = str(meeting_id) + str(agenda_id)
        total_id.append(id)

        if full_df['title'].iloc[i] == full_df['title'].iloc[i+1]:
            agenda_id += 1
        else:        
            meeting_id += 1
            agenda_id = 1

    # 마지막 agenda 더하기
    if len(str(agenda_id)) == 1:
        id = str(meeting_id) + '0' + str(agenda_id)
    else:
        id = str(meeting_id) + str(agenda_id)    
    total_id.append(id)
    
    return total_id

In [9]:
total_id = get_total_id(train)
train['total_id'] = total_id

print(len(total_id))
print(total_id[0])
print(total_id[-1])
print(total_id[-10:])

2994
100001
146210
['146201', '146202', '146203', '146204', '146205', '146206', '146207', '146208', '146209', '146210']


In [11]:
# 본문에 title 추가하기
train['total'] = '@' + train.title + '@' + '\n\n' + train.context

train.head()

,uid,title,region,context,summary,total_id,total
1000,1000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.,100001,@제207회 완주군의회(임시회) 제 1 차 본회의회의록@\n\n의석을 정돈하여 주시...
1001,1001,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.,100002,@제207회 완주군의회(임시회) 제 1 차 본회의회의록@\n\n의사팀장 수고하셨습니...
1002,1002,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.,100003,@제207회 완주군의회(임시회) 제 1 차 본회의회의록@\n\n다음은 의사일정 제2...
1003,1003,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...,100004,@제207회 완주군의회(임시회) 제 1 차 본회의회의록@\n\n다음은 의사일정 제3...
1004,1004,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.,100101,@제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록@\n\n의석을 ...


### 지역별로 나누기

In [13]:
region = train['region']
region = list(set(region))
region

['청주', '음성', '완주']

In [14]:
grouped = train.groupby('region')
grouped.count()

,uid,title,context,summary,total_id,total
region,,,,,,
완주,861,861,861,861,861,861
음성,1938,1938,1938,1938,1938,1938
청주,195,195,195,195,195,195


In [15]:
region_df1 = train[train['region'] == '완주']
region_df2 = train[train['region'] == '음성']
region_df3 = train[train['region'] == '청주']

print(len(region_df1))
print(len(region_df2))
print(len(region_df3))

861
1938
195


In [17]:
# 완주군 회의 개수
region_df1.groupby('title').count()

,uid,region,context,summary,total_id,total
title,,,,,,
제101회 전라북도 완주군의회 (임시회) 제1호 본 회 의 회 의 록,6,6,6,6,6,6
제102회 전라북도 완주군의회 (임시회) 제1호 본 회 의 회 의 록,5,5,5,5,5,5
제103회 전라북도 완주군의회 (임시회) 제1호 본 회 의 회 의 록,6,6,6,6,6,6
제105회 전라북도 완주군의회 (임시회) 제2호 본 회 의 회 의 록,3,3,3,3,3,3
제106회 전라북도 완주군의회 (임시회) 제1호 본 회 의 회 의 록,5,5,5,5,5,5
...,...,...,...,...,...,...
제96회 전라북도 완주군의회 (임시회) 제1호 본 회 의 회 의 록,6,6,6,6,6,6
제98회 전라북도 완주군의회 (임시회) 제1호 본 회 의 회 의 록,5,5,5,5,5,5
제98회 전라북도 완주군의회 (임시회) 제2호 본 회 의 회 의 록,4,4,4,4,4,4


### uid와 context만 뽑아 태깅 문서로 저장 (화자 정보는 날아감) - train data

In [20]:
def transform_df(train):
    train_doc = pd.DataFrame()
    train_doc['document_identifier'] = train['total_id']
    train_doc['context'] = train['context']
    train_doc['document_text'] = train['total']
    print("train data 개수: ", len(train_doc))
    
    return train_doc

In [21]:
# 지역별 데이터 저장

save_file1 = "../data/raw_data/region_doc_완주.csv"
trans_df1 = transform_df(region_df1)
trans_df1.to_csv(save_file1, header=True, index=False)

save_file2 = "../data/raw_data/region_doc_음성.csv"
trans_df2 = transform_df(region_df2)
trans_df2.to_csv(save_file2, header=True, index=False)


save_file3 = "../data/raw_data/region_doc_청주.csv"
trans_df3 = transform_df(region_df3)
trans_df3.to_csv(save_file3, header=True, index=False)

train data 개수:  861
train data 개수:  1938
train data 개수:  195


In [22]:
trans_df1.shape

(861, 3)